In [1]:
# Packages
import requests
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [3]:
# Make a map of league number code to league name
url = f'https://fbref.com/en/comps/0/history/'
page = requests.get(url)
#header = page.text.split('</h1>')[0].split('<h1>')[-1]
#' '.join(header.split('Seasons')[0].split('\t')[-1].split())
soup = BeautifulSoup(page.content, 'lxml')
while soup.find('tr', class_='thead') is not None:
    soup.find('tr', class_='thead').decompose()
table = soup.find('table', id = 'comps_1_fa_club_league_senior')
comps = pd.read_html(str(table))[0]
comps['CODE'] = [int(x['href'].split('/')[3]) for x in list(table.find_all('a', href = True)) if 'comps' in x['href']][0::3]
comps['fs_href'] = [x['href'] for x in list(table.find_all('a', href = True)) if 'comps' in x['href']][1::3]
comps = comps.groupby('Gender').get_group('M')
comps['Country'] = comps['Country'].apply(lambda x: x.split()[-1])
comps['First Season'] = comps['First Season'].apply(lambda x: x[-4:]).astype(int)
comps = comps.rename(columns = {'Competition Name': 'Competition'}).reset_index(drop = True)
comps['title'] = comps['Country'] + ' ' + comps['Competition'].apply(lambda x: x.lower())
comps = comps[['Competition', 'title', 'Country', 'First Season', 'CODE', 'fs_href']]
#comps.to_csv('competitions.csv', index = None)

In [35]:
url = 'https://fbref.com/en/comps/21/history/'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'lxml')
table = soup.find('table')
df = pd.read_html(str(table))[0]

In [36]:
df['href'] = [x['href'] for x in table.find_all('a', href = True) if 'comps' in x['href']][0::2]

In [37]:
df['Year'] = df['Season'].apply(lambda x: x.split('-')[-1])
df = df[['Season', 'Year', 'href']]

In [25]:
header = page.text.split('</h1>')[0].split('<h1>')[-1] # Find the header of the page
title = header.split(' ')[-1].split('\n')[0] # Find the type of data the page contains

In [43]:
fix = []
for href in df['href'] :

    url = 'https://fbref.com' + '/'.join(href.split('/')[:-1]) + '/schedule/'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'lxml')
    header = page.text.split('</h1>')[0].split('<h1>')[-1] # Find the header of the page
    title = header.split(' ')[-1].split('\n')[0] # Find the type of data the page contains

    if soup.find('table') is not None and title == 'Fixtures':

        fix.append(1)

    else:
        fix.append(0)


In [44]:
df['fix'] = fix
df

,Season,Year,href,fix
0,2023,2023,/en/comps/21/Primera-Division-Stats,1
1,2022,2022,/en/comps/21/2022/2022-Primera-Division-Stats,1
2,2021,2021,/en/comps/21/2021/2021-Primera-Division-Stats,1
3,2019-2020,2020,/en/comps/21/2019-2020/2019-2020-Primera-Divis...,1
4,2018-2019,2019,/en/comps/21/2018-2019/2018-2019-Primera-Divis...,1
5,2017-2018,2018,/en/comps/21/2017-2018/2017-2018-Primera-Divis...,1
6,2016-2017,2017,/en/comps/21/2016-2017/2016-2017-Primera-Divis...,1
7,2016,2016,/en/comps/21/2016/2016-Primera-Division-Stats,1
8,2015,2015,/en/comps/21/2015/2015-Primera-Division-Stats,1
9,2014,2014,/en/comps/21/2014/2014-Primera-Division-Stats,0


In [52]:
from datetime import datetime, timedelta

datetime(datetime.today().year, datetime.today().month, datetime.today().day)

datetime.datetime(2023, 5, 2, 0, 0)

In [47]:
datetime.today().strftime('%Y-%m-%d') + timedelta(days = 2)

TypeError: can only concatenate str (not "datetime.timedelta") to str

In [53]:
dic = {0:1, 2:3}

In [58]:
for a, b in dic.items():
    print(a, b)

0 1
2 3
